# Forte Tutorial 1.03: Forte's sparse operator class
---

Forte exposes several functions to create and manipulate general second quantized operators and wave functions.
In this tutorial we will look at simple examples that illustrate how these classes work.

## Preliminaries
Here we define a useful function to display equations in LaTeX format

In [16]:
import math
import forte
from IPython.display import display, Math, Latex

def latex(obj):
    """Call the latex() function on an object and display the returned value in LaTeX"""
    display(Math(obj.latex()))

In a previous tutorial we looked at how to define determinants in forte. Here we are going to use the utility function `forte.det()`, which creates a determinant from a string representation of the determinant. The occupation of each orbital is specified by the symbols `2` (doubly occupied), `+` (single alpha electron), `-` (single beta electron), `0` (empty).

Here are some examples.

In [17]:
print(forte.det('22+-'))
print(forte.det('22ba'))
print(forte.det('ABBA'))

|22+-000000000000000000000000000000000000000000000000000000000000>
|22-+000000000000000000000000000000000000000000000000000000000000>
|+--+000000000000000000000000000000000000000000000000000000000000>


Depending on the size if the `Determinant` class, these commands will return a 64 bit or longer representation of the determinants.

## The StateVector class

Sparse collections of determinants can be manipulated using the `StateVector` class. The simplest way to create a `StateVector` object is by passing a dictionary of `determinants -> double`. For example, here we create a superposition of a determinant with two electrons and one that has no electrons, both with equal coefficients normalized to one
$$
|\Psi\rangle = \frac{1}{\sqrt{2}}\left( |20\rangle + |00\rangle \right)
$$

In [18]:
c = 1./ math.sqrt(2.0)
psi = forte.StateVector({ forte.det('20'): c, forte.det('00') : c})
print(psi)

|2000000000000000000000000000000000000000000000000000000000000000> * 0.707107
|0000000000000000000000000000000000000000000000000000000000000000> * 0.707107



An alternative way to print this wave function is by calling the `str` method on the `StateVector` object. The argument `2` here indicates that we want to show only the occupation numbers of only the first two orbitals.

In [19]:
print(psi.str(2))

|20> * 0.707107
|00> * 0.707107



## The `SparseOperator` class

The `SparseOperator` class can handle operators of the form
$$
\hat{O} = \sum_{pqrs\cdots} t_{pq\cdots}^{rs\cdots} \hat{a}^\dagger_p \hat{a}^\dagger_q \cdots \hat{a}_s \hat{a}_r
$$
where each individual term in the summation can be an arbitrary order operator.
However, the amplitudes are assumed to be **real numbers**.

At creation, the user can specify if this operator should be anti-Hermitian, that is if each term should be paired with minus its Hermitian conjugate
$$
\hat{O} = \sum_{pqrs\cdots} t_{pq\cdots}^{rs\cdots} \left( \hat{a}^\dagger_p \hat{a}^\dagger_q \cdots \hat{a}_s \hat{a}_r 
- \hat{a}^\dagger_r \hat{s}^\dagger_q \cdots \hat{a}_q \hat{a}_p \right)
$$

### Creating `SparseOperator` objects

After creation, a `SparseOperator` object is empty
```python
op = forte.SparseOperator()
latex(op)
# displays nothing
```

The simplest way to populate a `SparseOperator` is by adding one term at a time using the `add_term_from_str` function.

A generic operator
$$
\hat{q}_1 \hat{q}_2 \cdots, \quad \text{ with } \hat{q}_i \in \{ \hat{a}_p, \hat{a}^\dagger_p\}
$$
can be specified using the following syntax
```
add_term_from_str('[<orbital_1><spin_1><type_1> <orbital_2><spin_2><type_2> ...]', amplitude)
```
where
```
orbital_i: int
spin_i: 'a' (alpha) or 'b' (beta)
type_i: '+' (creation) or '-' (annihilation)
```

For example, the operator $\hat{a}^\dagger_{1_\alpha} \hat{a}_{0_\alpha}$ is encoded as `[1a+ 0a-]`. The following code generates the operators $\hat{a}^\dagger_{1_\alpha} \hat{a}_{0_\alpha}$ and  $\frac{1}{2} (\hat{a}_{0_\alpha} - \hat{a}^\dagger_{0_\alpha})$

In [20]:
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[1a+ 0a-]',1.0)
latex(op)

op = forte.SparseOperator(antihermitian=True)
op.add_term_from_str('[0a-]',0.5)
latex(op)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Ordering of operators in the `SparseOperator` object

<div class="alert alert-block alert-warning">
Note that `add_term_from_str` <b>assumes that the operators will match a specific order!</b>

This canonical order is defined as
$$
(\alpha \text{ creation}) (\beta \text{ creation}) (\beta \text{ annihilation}) (\alpha \text{ annihilation})
$$
with the creation (annihilation) operators ordered within each group in increasing (decreasing) order.
The following operator satisfies the canonical order:
$$
+\;\hat{a}_{2 \alpha}^\dagger\hat{a}_{3 \alpha}^\dagger\hat{a}_{2 \beta}^\dagger\hat{a}_{3 \beta}^\dagger\hat{a}_{1 \beta}\hat{a}_{0 \beta}\hat{a}_{1 \alpha}\hat{a}_{0 \alpha}
$$
</div>

If you want to work with operators that do not follow this ordering, for example, $\hat{a}_{1 \alpha}\hat{a}^\dagger_{0 \alpha}$, you will need to work out an equivalent representation, for example, $\hat{a}_{0 \alpha}\hat{a}^\dagger_{0 \alpha} = 1 - \hat{a}^\dagger_{0 \alpha}\hat{a}_{0 \alpha}$.

These examples illustrate valid operators in canonical order

In [21]:
# beta annihilation operators appear to the left of alpha annihilation
# within each group, orbital indices decrease going from left to right
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[1b- 0b- 1a- 0a-]',1.0)
latex(op)

# beta creation operators appear to the right of alpha annihilation
# within each group, orbitals increase going from left to right
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0a+ 1a+ 0b+ 1b+]',1.0)
latex(op)

# creation operators appear to the left of annihilation operators
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[2a+ 3a+ 2b+ 3b+ 1b- 0b- 1a- 0a-]',1.0)
latex(op)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

When the operator passed is out of order, an exception is thrown. For example, the following code
```python
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0b- 1b- 1a- 0a-]',1.0)
latex(op)
```
leads to the following RuntimeError

In [22]:
op = forte.SparseOperator(antihermitian=False)
try:
    op.add_term_from_str('[0b- 1b- 1a- 0a-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

RuntimeError: Trying to initialize a SQOperator object with a product of
operators that are not arranged in the canonical form

    a+_p1 a+_p2 ...  a+_P1 a+_P2 ...   ... a-_Q2 a-_Q1   ... a-_q2 a-_q1
    alpha creation   beta creation    beta annihilation  alpha annihilation

with indices sorted as

    (p1 < p2 < ...) (P1 < P2 < ...)  (... > Q2 > Q1) (... > q2 > q1)



This error can be overriden. However, **this is recommended only if you understand what happens when you do so**. The function `add_term_from_str` has an extra option that allows it to reorder the operators to the canonical form. The final operator is multiplied by a sign factor that corresponds to the parity of the permutation that connects the initial and final ordering. This code illustrates how this reordering happens

In [23]:
# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# The operator [0a- 0b- 1a- 1b-] (see above) is equivalent to -[1a- 1b- 0b- 0a-].
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[1a- 1b- 0b- 0a-]',-1.0,allow_reordering=True)
latex(op)

# Another example that illustrates the reordering of operators
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0a- 0b- 1a- 1b- 2a+ 2b+ 3a+ 3b+]',1.0,allow_reordering=True)
latex(op)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

An exception is also thrown if two operators are repeated. For example, the following code
```python
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[0b- 0b-]',1.0)
```
gives to the following RuntimeError

In [24]:
op = forte.SparseOperator(antihermitian=False)
try:
    op = forte.SparseOperator(antihermitian=False)
    op.add_term_from_str('[0b- 0b-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

RuntimeError: Trying to initialize a SQOperator object with a product of
operators that contains repeated operators.



### Specifying a full operator with the `SparseOperator` class

To form a full operator we can just keep adding terms to a `SparseOperator` object. For example

In [25]:
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[1a+ 0a-]',0.3)
op.add_term_from_str('[1b+ 0b-]',0.3)
op.add_term_from_str('[1a+ 1b+ 0b- 0a-]',0.1)
latex(op)

<IPython.core.display.Math object>

Another way to create an operator is via the function `add_term` by providing a list of tuples that specifies the second quantized operators and the corresponding amplitude. This is useful when building operators with a large number of terms. Note, that **this function uses a different convention than `add_term_from_str` for expressing the ordering of the operators**. Here we specify the operator (in reversed order)
$$
\cdots \hat{q}_2 \hat{q}_1, \quad \text{ with } \hat{q}_i \in \{ \hat{a}_p, \hat{a}^\dagger_p\}
$$
with the following syntax
```
add_term([(type_1, spin_1, orb_1), (type_2, spin_2, orb_2), ...]', amplitude)
```
where
```
type_i: bool (true = creation, false = annihilation)
spin_i: bool (true = alpha, false = beta)
orb_i: int
```
For example, the operator $\hat{a}^\dagger_{1_\alpha}\hat{a}_{0_\alpha}$ is generated in this way

In [26]:
op = forte.SparseOperator()
op.add_term([(False,True,0),(True,True,1)],1.0)
op.str()
latex(op)

<IPython.core.display.Math object>

## Applying a `SparseOperator` to a `StateVector`

To apply an operator to a state vector you can use the `forte.apply_operator(op,psi)` function. This function takes an operator (`op`) and a state (`psi`), and returns the state `|new_psi> = op |psi>`. For example, the following creates a CIS wave function using the operator
$$
\hat{T} = 0.1\; +0.3 \left(\hat{a}_{1 \alpha}^\dagger\hat{a}_{0 \alpha} + \hat{a}_{1 \beta}^\dagger\hat{a}_{0 \beta} \right)
$$
where the first term is just a scalar

In [27]:
op = forte.SparseOperator(antihermitian=False)
op.add_term_from_str('[]',0.1)
op.add_term_from_str('[1a+ 0a-]',0.3)
op.add_term_from_str('[1b+ 0b-]',0.3)
psi = forte.StateVector({ forte.det('2'): 1.0})
new_psi = forte.apply_operator(op,psi)
print(new_psi.str(3))

|+-0> * 0.300000
|-+0> * 0.300000
|200> * 0.100000



### Exponential operator

To apply the exponential operator $\exp(\hat{T})$ we can use the class `SparseExp` class. This class provides the method `compute` which takes as arguments the operator and the state

In [28]:
psi = forte.StateVector({ forte.det('2'): 1.0})
exp_op = forte.SparseExp()
new_psi = exp_op.compute(op,psi)
print(new_psi.str(3))

|020> * 0.099465
|-+0> * 0.331551
|+-0> * 0.331551
|200> * 1.105171



There are several variables that control the behavior of `compute`. For example, to compute the inverse, we can just apply $\exp(-\hat{T})$

In [29]:
new_psi2 = exp_op.compute(op,new_psi,scaling_factor=-1.0)
print(new_psi2.str(3))

|200> * 1.000000



By default `compute` uses a caching algorithm that reuses information from previous applications of the exponential. A memory-light algorithm can be also invoked

In [30]:
psi = forte.StateVector({ forte.det('2'): 1.0})
new_psi = exp_op.compute(op,psi,algorithm='onthefly')
print(new_psi.str(3))

|020> * 0.099465
|-+0> * 0.331551
|+-0> * 0.331551
|200> * 1.105171



We can also control other parameters, like the order of the Taylor expansion used to approximate $\exp(\hat{T})$ (`maxk`) and a threshold used to screen term (`screen_thresh`). For example, to apply $1 + \hat{T}$ we can call

In [31]:
psi = forte.StateVector({ forte.det('2'): 1.0})
new_psi = exp_op.compute(op,psi,algorithm='onthefly',maxk=1)
print(new_psi.str(3))

|-+0> * 0.300000
|+-0> * 0.300000
|200> * 1.100000



Note that the most efficient algorithm to compute the exponential of an operator via `SparseExp`
assumes that the function is always called with the same operator.
For example, if `op1` and `op2` are two different `SparseOperator` obects, the following code will give
an incorrect result
```python
exp_op = forte.SparseExp()
psi1 = exp_op.compute(op1,psi0)
psi2 = exp_op.compute(op2,psi1)
```
However, if we ask the `SparseExp` class to use an on-the-fly algorithm via the following code
```python
exp_op = forte.SparseExp()
psi1 = exp_op.compute(op1,psi0,algorithm='onthefly')
psi2 = exp_op.compute(op2,psi1,algorithm='onthefly')
```
then the result will be correct.

### Factorized exponential of an anti-Hermitian operator

Another useful operator is the factorized exponential of an operator $\hat{T}$. If $\hat{T}$ is a sum of operators
$$
\hat{T} = \sum_\mu t_\mu \hat{\kappa}_\mu
$$
the factorized exponential is defined as
$$
\exp_\mathrm{f}(\hat{T}) = \prod_\mu \exp(t_\mu \hat{\kappa}_\mu)
$$
This operation is implemented in the class `SparseFactExp` for the case of anti-Hermitian operators, that is, when $(\hat{T})^\dagger = - \hat{T}$.
This class provides the method `compute` which takes as arguments the operator and the state. Here is a simple example:

In [32]:
op = forte.SparseOperator(antihermitian=True)
op.add_term_from_str('[1a+ 0a-]',0.3)
op.add_term_from_str('[1b+ 0b-]',0.3)

psi = forte.StateVector({ forte.det('2'): 1.0})
factexp_op = forte.SparseFactExp()
new_psi = factexp_op.compute(op,psi)
print(new_psi.str(3))

|+-0> * 0.282321
|020> * 0.087332
|-+0> * 0.282321
|200> * 0.912668



To compute the inverse of the factorized exponential, just pass the option `inverse=True` to `compute()`:

In [33]:
starting_psi = factexp_op.compute(op,new_psi,inverse=True)
print(starting_psi.str(3))

|200> * 1.000000

